In [1]:
import tensorflow as tf
import numpy as np

from ipynb.fs.full.create_sentiment_featuresets import create_feature_sets_and_labels

train_x, train_y, test_x, test_y = create_feature_sets_and_labels('pos.txt', 'neg.txt')

n_nodes_1 = 500
n_nodes_2 = 500
n_nodes_3 = 500

n_classes = 2
batch_size = 100

# height * width
x = tf.placeholder('float',[None, len(train_x[0])])
y = tf.placeholder('float')

424


In [2]:
def neural_network_model(data):
    
    # Setting up the computation graph, means arrange how the model will work
    hidden_1_layer = {
        'weights': tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_1])),
        'bias': tf.Variable(tf.random_normal([n_nodes_1])),
    }

    hidden_2_layer = {
        'weights': tf.Variable(tf.random_normal([n_nodes_1, n_nodes_2])),
        'bias': tf.Variable(tf.random_normal([n_nodes_2])),
    }

    hidden_3_layer = {
        'weights': tf.Variable(tf.random_normal([n_nodes_2, n_nodes_3])),
        'bias': tf.Variable(tf.random_normal([n_nodes_3])),
    }

    output_layer = {
        'weights': tf.Variable(tf.random_normal([n_nodes_3, n_classes])),
        'bias': tf.Variable(tf.random_normal([n_classes])),
    }

    # (input_data * weight) + biases

    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['bias'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3, output_layer['weights']) + output_layer['bias']

    return output

In [9]:
def train_neural_network(x):
    
    print(x, y)
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    hm_epochs = 10

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            i = 0
            while i < len(train_x):
                start = i
                end = i+batch_size
                
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict = {x:batch_x, y:batch_y})
                epoch_loss += c
                i += batch_size
                
            print('Epoch', epoch + 1, 'complete out of', hm_epochs, 'loss:', epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print("Accuracy", accuracy.eval({x:test_x, y:test_y}))

In [10]:
train_neural_network(x)

Tensor("Placeholder:0", shape=(?, 424), dtype=float32) Tensor("Placeholder_1:0", dtype=float32)
Epoch 1 complete out of 10 loss: 215555.81665039062
Epoch 2 complete out of 10 loss: 104940.4067993164
Epoch 3 complete out of 10 loss: 63972.50115966797
Epoch 4 complete out of 10 loss: 39786.459869384766
Epoch 5 complete out of 10 loss: 25288.608116149902
Epoch 6 complete out of 10 loss: 16685.432006835938
Epoch 7 complete out of 10 loss: 17104.528198242188
Epoch 8 complete out of 10 loss: 19656.711603164673
Epoch 9 complete out of 10 loss: 15366.340913772583
Epoch 10 complete out of 10 loss: 9568.344597816467
Accuracy 0.5947467
